In [1]:
from pyincore import IncoreClient
from pyincore.analyses.cumulativebuildingdamage.cumulativebuildingdamage import CumulativeBuildingDamage

In [2]:
# Connect to IN-CORE service 
client = IncoreClient()

Connection successful to IN-CORE services. pyIncore version detected: 0.9.0


### Cumulative Building Damage Analysis for Seaside, OR

In [3]:
# Setting up Id's for Dataset inputs: 
# Earthquake building damage and Tsunami Building Damage
eq_bldg_dmg_id = "5c5c9686c5c0e488fcf91903"
tsunami_bldg_dmg_id = "5c5c96f6c5c0e488fcf9190f" 

In [4]:
# Create cumulative Building Damage
cumulative_bldg_dmg = CumulativeBuildingDamage(client)

# Load input datasets
cumulative_bldg_dmg.load_remote_input_dataset("eq_bldg_dmg", eq_bldg_dmg_id)
cumulative_bldg_dmg.load_remote_input_dataset("tsunami_bldg_dmg", tsunami_bldg_dmg_id)

# Specify the result name
result_name = "cumulative_bldg_dmg_result"

# Set analysis parameters
cumulative_bldg_dmg.set_parameter("result_name", result_name)
cumulative_bldg_dmg.set_parameter("num_cpu", 4)

dataset type does not match - given type: ergo:buildingDamageVer4 spec types: ['ergo:buildingDamageVer5']
dataset type does not match - given type: ergo:buildingDamageVer4 spec types: ['ergo:buildingDamageVer5']


True

In [5]:
# Run Cumulative Building Damage Analysis
cumulative_bldg_dmg.run_analysis()

Error reading dataset: required dataset is missing - spec: {'id': 'eq_bldg_dmg', 'required': True, 'description': 'Earthquake Building Damage Results', 'type': ['ergo:buildingDamageVer5']}


(False,
 "required dataset is missing - spec: {'id': 'eq_bldg_dmg', 'required': True, 'description': 'Earthquake Building Damage Results', 'type': ['ergo:buildingDamageVer5']}")

In [6]:
# Retrieve result dataset
result = cumulative_bldg_dmg.get_output_dataset("combined-result")

# Convert dataset to Pandas DataFrame
df = result.get_dataframe_from_csv()

# Display top 5 rows of output data
df.head()

AttributeError: 'NoneType' object has no attribute 'get_dataframe_from_csv'